# Lesson 7 - Sequentially Chaining the Agent Calls

Now that you have two activated ACP servers, you will create a sequential workflow by sequentially chaining the two agents as follows:
- the Hospital agent that handles health-based questions for patients will execute first;
- the Insurer agent that handles questions related to insurance coverage will execute secondly; 
- since these two agents are hosted on two separate servers, you will need two clients: each client will interact with one server.
  - the first client will send the user's query to the hospital server and receives the response from the server;
  - you will pass the response from the first client to the second client which will combine in its query and send it to the second server 

<img src="seq.png" width="650">

## 7.1. Run the Insurer ACP Server

First make sure the Insurer server is still running:
- Open the terminal by running the cell below.
- If the agent is still running from the previous lessons, then you don't need to do anything else.
- If the agent has stopped running (the lab environment resets after 120 min), then you can run the server again by typing:
  - `uv run crew_agent_server.py`

## 7.2. Run the Hospital ACP Server

Also make sure the Hospital server is still running:
- Open the terminal by running the cell below.
- If the agent is still running from the previous lessons, then you don't need to do anything else.
- If the agent has stopped running (the lab environment resets after 120 min), then you can run the server again by typing:
  - `uv run smolagents_server.py`


**Note**: If you see this warning: 
`WARNING: Can not reach server, check if running on http://127.0.0.1:8333 : Request failed after 5 retries`
you can ignore it. You'll learn later in another lesson about the BeeAI platform, which a registry you can use to manage and discover agents. If the platform is installed, it runs by default on port 8333. The ACP servers are configured to automatically connect to the platform. Since the platform is not installed in this environment, the ACP server will generate a warning.

## 7.3. Chain the LLM Calls 

You will now create two ACP clients, each will interact with one agent so that hospital agent will execute first and the insurer agent will execute secondly.

In [1]:
import nest_asyncio

nest_asyncio.apply()

In [2]:
import asyncio

from acp_sdk.client import Client
from colorama import Fore


async def run_hospital_workflow() -> None:
    async with (
        Client(base_url="http://localhost:8000") as hospital,
        Client(base_url="http://localhost:8001") as insurer,
    ):
        run1 = await hospital.run_sync(
            agent="health_agent",
            input="Do I need rehabilitation after a shoulder reconstruction?",
        )
        content = run1.output[0].parts[0].content
        print(Fore.LIGHTMAGENTA_EX + (content or "N/A") + Fore.RESET)

        run2 = await insurer.run_sync(
            agent="policy_agent",
            input=f"Context: {content} What is the waiting period for rehabilitation?",
        )
        content2 = run2.output[0].parts[0].content
        print(Fore.YELLOW + (content2 or "N/A") + Fore.RESET)

In [3]:
asyncio.run(run_hospital_workflow())

Yes, rehabilitation is typically required after shoulder reconstruction surgery, as it is essential for recovering joint mobility, muscle strength, and overall shoulder function.
According to the provided knowledge base, the waiting period for rehabilitation after shoulder reconstruction surgery is 2 months.


## 7.4. Resources

- [Compose Agents](https://agentcommunicationprotocol.dev/how-to/compose-agents)
- [Chained Agents as a Python file](https://github.com/nicknochnack/ACPWalkthrough/blob/main/5.%20Chained%20Agents.py)